## Ant Colony Optimization - Monte Carlo Hybrid for the Set Covering Problem

The *Set Cover Problem*, consist of a given set of *m* elements *A={a1,a2,...,am}*. A set of *n* subsets of A, defined as *F={A1,A2,...,An} where A1,...,An are subsets of A*. And a weight function (cost) *F*. The objective of the problem is to find a set *I={1,2,...,k} with k lower or equals than n*, that minimizes the sum of costs for each set, and such that the union of subsets *Fi* cover the whole amount of *n* elements.

The idea is to use a hybrid solution of Markov Chain Monte Carlo method and Ant Colony Optimization to find a solution to the problem. As python is not expected to have outstanding performance, the idea is to show the main ideas to be implemented in a compiled language and explore the main parts of the algorithm using the explanations given in the jupyter notebook.

The main ideas for the solution are based on two publications: *New ideas for applying ant colony optimization to the set covering problem*, Ren, Feng et al. 2010 and *A Hybrid Monte Carlo Ant Colony Optimization Approach for Protein Structure Prediction in the HP Model*, Citrolo, Mauri, 2013.

The set of files used for testing can be found in the resources folder of the project (./resources)

In [260]:
import numpy as np
import random
import math
import operator
import copy

random.seed(1)


### Problem Class:
Definition of the problem class which will contain the data and methods related to a problem. Each object will be an instance of a problem that will be used by each ant. The main information is loaded into a matrix where **rows**
represent the **elements** and **columns** are associated with the **sets** for the problem. This matrix contains boolean information which allows us to know whether a set contains an element.

**Redundancy Elimination:**
This is a process that helps to clean the solution. Given that in the process of assembling the solution some sets can be redundant, meaning that several sets can cover elements that are already covered by others, it is useful to run a cleaning process to avoid this situation. This process is called *Redundancy Elimination*. In this case we proceed to check each of the covered sets from higher to lower cost and we decide wether it is good to eliminate it or not.Given that the input files have the cost values for the sets ordered from lower to higher, it is enough to order the covered sets in a descending way to get the higher costs first and try to remove them in this way.

In [261]:
class Problem:
    
    # Definition of problem variables to be used in the solution
    num_elements = 0
    num_sets = 0
    sets_cost = None
    matrix = None
    
    covered_sets_monitor = None
    covered_elements_monitor = None
    
    
    # Helper function to add flag elements to the reference matrix
    def add_elements(self, mat, idx, elems):
        for e in elems:
            mat[idx,(e-1)] = 1
        return mat

    # Definition of the function in charge to read the instance file.
    def read_file(self, fname):
        with open(fname) as f:
            content = f.readlines()
        self.num_elements, self.num_sets = content[0].split()
        self.num_elements = int(self.num_elements)
        self.num_sets = int(self.num_sets)

        self.matrix = np.zeros((self.num_elements,self.num_sets), dtype=np.int)

        line_idx = 1

        sets_cost_tmp = []
        
        while(len(sets_cost_tmp) < self.num_sets):
            sets_cost_tmp.extend([int(x) for x in content[line_idx].split()])
            line_idx += 1
        
        self.sets_cost = np.array(sets_cost_tmp)

        for r in range(0, self.num_elements):
            tmp_list=[]
            n_els = int(content[line_idx])
            line_idx += 1
            while(len(tmp_list) < n_els):
                tmp_list.extend([int(x) for x in content[line_idx].split()])
                line_idx += 1
            self.matrix = self.add_elements(self.matrix, r, tmp_list)
            
        #Initialize required values after data load
        self.init_covered_lists()
        print("Problem initialized succesfully from file: " + fname)
    
    def init_covered_lists(self):
        self.covered_elements_monitor = np.zeros(self.num_elements, dtype=np.int)
        self.covered_sets_monitor = np.zeros(self.num_sets, dtype=np.int)
        
    def get_covered_sets(self):
        return np.where(self.covered_sets_monitor == 1)[0]
    
    def get_uncovered_elements(self):
        return np.where(self.covered_elements_monitor == 0)[0]
    
    def get_uncovered_sets(self):
        return np.where(self.covered_sets_monitor == 0)[0]
    
    def get_covered_elements(self):
        return np.where(self.covered_elements_monitor == 1)[0]
    
    def covered_sets_cost(self):        
        return np.sum(self.sets_cost[self.get_covered_sets()])
    
    #Returns the list of sets that contain the element given as parameter
    def sets_for_element(self, elem):
        return np.where(self.matrix[elem,:] == 1)[0]
    
    def elements_for_set(self, sett):
        return np.where(self.matrix[:,sett] == 1)[0]
    
    #Returns the list of available sets to be possibly covered for an element
    def available_sets_for_element(self, elem):
        return np.setdiff1d(self.sets_for_element(elem), self.get_covered_sets())
    
    #Returns the list of elements available to be covered for a set
    def available_elements_for_set(self, sett):
        return np.setdiff1d(self.elements_for_set(sett), self.get_covered_elements())
    
    #Covers a set covering at the same time the elements associated to it
    def cover_set(self, sett):
        self.covered_sets_monitor[sett] = 1
        self.covered_elements_monitor[self.elements_for_set(sett)] = 1
            
    #Logic to cover an element. Add and remove the element to cover to/from the reference lists
    def cover_element(self, elem):
        self.covered_elements_monitor[elem] = 1
    
    #Method in charge of the redundancy elimination process.
    #Goes through the covered sets order by cost and tries to remove
    #each set if it is redundant
    def redundancy_elimination(self):
        for s in list(reversed(sorted(self.get_covered_sets()))):
            self.uncover_set_if_redundant(s)
    
    def uncover_set_if_redundant(self, sett):
        cov_sets = list(self.get_covered_sets())
        cov_sets.remove(sett)
        if len(self.get_covered_elements()) == len(self.get_covered_elements_for_list(cov_sets)):
            self.covered_sets_monitor[sett] = 0
    
    #Returns a list of covered elements for the input sets
    def get_covered_elements_for_list(self, sets_list):
        cov_elems = []
        for s in sets_list:
            cov_elems.extend(self.elements_for_set(s))
        return list(set(cov_elems))
    
    #Attempt to vectorize method. Not working
    def get_covered_elements_for_listt(self, sets_list):
        return np.union1d(self.elements_for_set(sets_list))

### Problem Solver Class:
This class is in charge of putting all things together, creating an initial instance for the problem, reading it from the input file and organizing the whole project to run the set of iterations with the parameters declared as variables. The set of parameters of the problem should be modified in this class. This class orchestrates the whole execution of the algorithm. First reads the problem from the file. After this, initializes the values for the pheromone to be used by the colony in the problem. Then, it creates a set of ants with the initial value for the heuristic information.

After setting up the context of the problem the *execute* method should be called to run the iterations for the problem and find a solution. This method runs a defined number of loops with the next structure: For the set of ants of the colony, a solution is found for each ant. The best ant of the batch is selected and then a *Metropolis Hastings condition* is evaluated to determine whether it is going to be used as the best global solution found or not. This metropolis condition helps increasing the exploration of the algorithm. Given its nature, exploration is increased at the beginning of the algorithm and is less when the algorithm is closer to finish. Last but not least, this class is in charge of updating the pheromone after each batch of ants run, this is very important given that this is the information shared by the whole colony to guide the improvement process.

In [262]:
class ProblemSolver:
    # Properties used for the solution
    problem = None
    ants = None
    pheromone = None
    best_ant = None
    min_pheromone = None
    max_pheromone = None
    number_of_ants = 30
    rho = 0.8
    epsilon = 0.01
    beta = 5.0
    t_0 = 8
    
    max_loops = 30
    
    def __init__(self, instance_file):
        self.problem = Problem()
        self.problem.read_file(instance_file)
        self.init_pheromone()
        self.init_ants(self.init_heuristic_information(), self.beta)
    
    # Method to initialize the pheromone values
    def init_pheromone(self):
        self.pheromone = np.empty(self.problem.num_sets)
        self.pheromone.fill(9999999)
        self.max_pheromone = 9999999
    
    # Initialize the array of ants with the values for the 
    # problem instance, pheromone, heuristic information and beta
    def init_ants(self,heuristic_info, b):
        self.ants = []
        for i in range(0,self.number_of_ants):
            self.ants.append(Ant(self.problem,self.pheromone, heuristic_info, b))
    
    # Initially the heuristic information for each set has the value of the 
    # number of elements divided by the cost of the set
    def init_heuristic_information(self):
        heuristic_info = np.zeros(self.problem.num_sets)
        for i in range(0, self.problem.num_sets):
            heuristic_info[i] = (np.sum(self.problem.matrix[:,i])/self.problem.sets_cost[i])
        return heuristic_info
    
    #Method in charge of the whole execution of the algorithm
    def execute(self):
        first_loop = True
        loop_counter = 0
        while not self.terminate(loop_counter):
            current_best_ant = None
            for ant in self.ants:
                ant.solve()
                # Apply redundancy elimination here
                if current_best_ant is None:
                    current_best_ant = ant
                elif current_best_ant.cost() > ant.cost():
                    current_best_ant = copy.deepcopy(ant)
                    #current_best_ant = Ant(ant.problem, ant.pheromone, ant.heuristic_information, ant.beta)
                
            if (self.best_ant is None 
                or (self.best_ant is not None 
                    and (self.metropolis_accept(loop_counter, self.best_ant, current_best_ant)))):
                self.best_ant = copy.deepcopy(current_best_ant)
                    
            self.update_pheromone(current_best_ant, first_loop)
            first_loop = False
            print("Loop: " + str(loop_counter) + ", cost: "+str(self.best_ant.cost()))
            loop_counter = loop_counter + 1
    
    #Method Used to update the values of the pheromone used by the ants. This
    #is invoked on each iteration done over the whole colony in order to
    #evaporate pheromone and update the importance of the values for the best
    #solution contained by the best ant.
    def update_pheromone(self, current_best_ant, first_loop):
        self.evaporate_pheromone()
        delta = 1.0/self.best_ant.cost()
        
        #Update max and min pheromone values when a new set solution is found
        if set(current_best_ant.problem.get_covered_sets()) == set(self.best_ant.problem.get_covered_sets()):
            self.max_pheromone = 1.0 / ((1.0 - self.rho)*self.best_ant.cost())
            self.min_pheromone = self.epsilon * self.max_pheromone
        
        #punctual update of pheromone values
        for s in self.best_ant.problem.get_covered_sets():
            pher_val = self.rho * self.pheromone[s] + delta
            if pher_val < self.max_pheromone and pher_val > self.min_pheromone:
                self.pheromone[s] = pher_val
            elif pher_val > self.max_pheromone:
                self.pheromone[s] = self.max_pheromone
            elif pher_val < self.min_pheromone:
                self.pheromone[s] = self.min_pheromone
    '''
    # Attempt to vectorize method. Not working
    
        np.putmask(self.pheromone[self.best_ant.problem.get_covered_sets()],
                  self.pheromone[self.best_ant.problem.get_covered_sets()] * self.rho + delta < self.max_pheromone
                  and self.pheromone[self.best_ant.problem.get_covered_sets()] * self.rho + delta > self.min_pheromone,
                  self.pheromone[self.best_ant.problem.get_covered_sets()] * self.rho + delta)
        
        np.putmask(self.pheromone[self.best_ant.problem.get_covered_sets()],
                  self.pheromone[self.best_ant.problem.get_covered_sets()] * self.rho + delta > self.max_pheromone,
                  self.max_pheromone)
        
        np.putmask(self.pheromone[self.best_ant.problem.get_covered_sets()],
                  self.pheromone[self.best_ant.problem.get_covered_sets()] * self.rho + delta < self.min_pheromone,
                  self.min_pheromone)
                  
    ###
    
        self.pheromone[self.best_ant.problem.get_covered_sets()] = self.tmp(self.best_ant.problem.get_covered_sets(), delta)
        
    def tmp(self, x, delta):
        pher_val = self.rho * x + delta
        if pher_val < self.max_pheromone and pher_val > self.min_pheromone:
            return pher_val
        elif pher_val > self.max_pheromone:
            return self.max_pheromone
        elif pher_val <= self.min_pheromone:
            return self.min_pheromone
        '''       
    
    #Method in charge of the pheromone evaporation. Uses rho as the main
    #parameter to update the pheromone.
    def evaporate_pheromone(self):
        if self.min_pheromone is None:
            self.min_pheromone = 0.0
        np.putmask(self.pheromone, self.pheromone * self.rho < self.min_pheromone, self.min_pheromone)
        np.putmask(self.pheromone, self.pheromone * self.rho >= self.min_pheromone, self.pheromone * self.rho)
    
    #Evaluates if the termination condition is achieved
    def terminate(self, loop_count):
        return loop_count >= self.max_loops
    
    #Uses the metropolis hasting condition to evaluate whether accept
    #the values of the current_best_ant or not
    def metropolis_accept(self, loop, best_ant, current_best_ant):
        return random.uniform(0,1) < self.p_accept(self.t_0, 
                                       loop, 
                                       self.max_loops, 
                                       best_ant.cost(), 
                                       current_best_ant.cost())
    
    def p_accept(self, init_temp, loop, num_loops, current_cost, neighbour_cost):
        if neighbour_cost <= current_cost:
            return 1.0
        else:
            return math.exp((current_cost - neighbour_cost)/(init_temp - loop * (init_temp/num_loops)))
    

### Ant Class:
Definition of the Ant class. Ant's are the basic unit of an Ant Colony Optimization algorithm. In this case the ants contain three main data objects. One to store an instance of the problem. Another one to store the pheromone information and a last one for the heuristic information. In this case a variation of **Max-Min Ant System** is used. There is an important concept for these kind of algorithms known as the pheromone. This idea gives the colony a global communication method, so that ants can share information about their independent process and make the others aware of their findings, contributing to the general global objective.

In this solution the heurstic and pheromone information are defined around the sets, this is a key point that allows the use of an Ant Colony Optimization problem definition for the set covering problem, given that it is not very straightforward to find the conditions that allow using this algorithm for any kind of problem. Here each ant has to select a set to cover in each step giving the idea of a path followed by the ant. The method introduces monte carlo perturbations to set the probability of each set after every iteration over the colony.

The main idea behind the basic definition of Max-Min Ant System is to define an upper and lower limit for the pheromone values to give the algorithm a higher chance to escape from local minima. In this case the exploration is also enhanced by a simulated annealing process given by the Metropolis Hastings condition. In order to see the main method using the probability based on the uncovered sets and elements we can go to the method *cover_next_set* where the whole step by step covering process takes place. This reflects the proposal of *Ren, Feng et al.*

In [263]:
class Ant:
    
    # Main properties of the class
    problem = Problem()
    pheromone = None
    heuristic_information = None
    
    # MAKE SURE THIS PARAMETER IS GOING TO BE USED. SUPOSEDLY USED IN THE PROBABILITY TO PICK A SET IN THE CONSTRUCTION
    beta = None
    
    def __init__(self, prob, pher, heur_inf, b):
        self.problem = copy.copy(prob)
        self.pheromone = copy.copy(pher)
        self.heuristic_information = copy.copy(heur_inf)
        self.beta = b
    
    def solve(self):
        self.problem.init_covered_lists()
        while self.problem.get_uncovered_elements().size > 0:
            self.cover_next_set()
        self.problem.redundancy_elimination()
    
    def cover_next_set(self):
        elem = random.choice(self.problem.get_uncovered_elements())
        #set_to_cover = random.choice(self.problem.available_sets_for_element(elem))
        set_to_cover = self.get_set_to_cover(self.problem.available_sets_for_element(elem))
        self.problem.cover_set(set_to_cover)
        self.update_heuristic_information(set_to_cover)
        
    #Gets a set to cover from the given list of sets
    def get_set_to_cover(self, sets):
        rnd = random.uniform(0,1)
        zum = 0.0
        for s in sets:
            zum = zum + self.probability_for_set(s, sets)
            if zum >= rnd:
                return s
        print("ERROR: This message should never be reached!")
    
    #Uses the probability condition based on the uncovered sets 
    #and possible elements to cover, in order to determine a probability
    #to choose a set
    def probability_for_set(self, sett, sets):
        numerator = self.pheromone[sett] * math.pow(self.heuristic_information[sett], self.beta)
        denominator = np.sum(self.pheromone[sets] * self.heuristic_information[sets]**self.beta)
        return numerator/denominator
        
    def update_heuristic_information(self, sett):
        sets_to_update = set()
        for elem in self.problem.available_elements_for_set(sett):
            sets_to_update |= set(self.problem.available_sets_for_element(elem))
        for s in sets_to_update:
            self.heuristic_information[s] = len(self.problem.available_elements_for_set(s))/self.problem.sets_cost[s]

    def weighted_cost(self, num_elements, cost):
        return num_elements/cost
    
    def cost(self):
        return self.problem.covered_sets_cost()
        

The following lines of code are the direct invocation to the problem solver creation and execution of the process.

In [264]:
solver =  ProblemSolver("./resources/scp-instances/scp42.txt")
solver.execute()


Problem initialized succesfully from file: ./resources/scp-instances/scp42.txt


ValueError: zero-dimensional arrays cannot be concatenated